# S.VI.B: FIMS

This notebook plots and analyzes the results shown in Figure 7a -- 7f of Section VI.B: "FIMS."

To simplify reproducability of the artifact, we have created three slightly-modified copies of the FIMS software pipeline presented in:

> D. Wang, J. Zhang, J. Buhler, and J. Wang, “Real-time analysis of aerosol size distributions with the fast integrated mobility spectrometer (FIMS),” in 41st Conference of American Association for Aerosol Research (AAAR), Oct. 2023. [Online]. Available: https://aaarabstracts.com/2023/view_abstract.php?pid=752 

These include:
* `fims_execution_time_profile`: Obtains execution times with `getrusage()` for each job iteration of all tasks.
* `fims_elasticity_values_assignment`: Obtains outputs using custom periods (stacking images, interpolating HK data, using different particle time bins for inversion) to assess impact of changing rates on result quality, allowing us to obtain elasticity constants.
* `fims_evaluate_scalability`: Runs FIMS using custom periods in a resource-constrained environment and measures job response times to assess whether all deadlines are met.

Other files include:
* `over_head`: A custom interference task to evaluate FIMS (`fims_evaluate_scalability`) in a simulated resource-constrained environment.
* `raw_data`: Input data (particle images captured from the FIMS camera, HK data captured from FIMS sensors) to test the pipeline.

## Build and Prepare System

### Enable `sudo` Commands from Jupyter Notebook

Several of the scripts in this notebook have to be run as `root`, e.g., to set real-time priorities. We use the `getpass` Python module to prompt for a password, then invoke `sudo` using `os.system`.

In [ ]:
import getpass
import os
password = getpass.getpass()
def run_sudo(cmd) :
    return os.system('echo %s | sudo -S %s' % (password, cmd))
if run_sudo("su") != 0 :
    print("Entered password incorrect! Please run this cell again.")


### Install OpenCV

***Note: The following script can be skipped on the provided virtual machine***

Refer to the OpenCV installation tutorial for more detailed instrutions:
https://docs.opencv.org/4.7.0/d7/d9f/tutorial_linux_install.html

In [ ]:
%%sh

# Download and unpack sources
wget -O opencv.zip https://github.com/opencv/opencv/archive/4.7.0.zip
wget -O opencv_contrib.zip https://github.com/opencv/opencv_contrib/archive/4.7.0.zip
unzip opencv.zip
unzip opencv_contrib.zip

# Create build directory and switch into it
mkdir -p build && cd build

# Configure
cmake -DOPENCV_EXTRA_MODULES_PATH=../opencv_contrib-4.7.0/modules ../opencv-4.7.0

# Build
make -j4

In [ ]:
# Install
run_sudo("make install")

### Compile FIMS Code

***Note: The following script can be skipped on the provided virtual machine***

In [ ]:
%%sh

# Compile Code for Profiling Execution Times
cd fims_execution_time_profile
mkdir -p build && cd build
cmake ..
make -j4

# Compile Code for Assigning Elasticity Values
cd ../../fims_elasticity_values_assignment
mkdir -p build && cd build
cmake ..
make -j4

# Compile Code for Evaluation of Online Adjustment
cd ../../fims_evaluate_scalability
mkdir -p build && cd build
cmake ..
make -j4

# Compile Code for Interference Task
cd ../../over_head
make

### Disable Throttling

CPU throttling and real-time scheduler throttling were disabled for our experiments. The following settings were used.

***Note that this is optional, and not required for correct functionality (though is required to faithfully reproduce timing results on a real system).***

In [ ]:
run_sudo("sh -c 'echo -1 > /proc/sys/kernel/sched_rt_runtime_us'")
run_sudo("cpufreq-set -r -g performance")

## Profiling Execution Times

We first profile the execution times of each FIMS pipeline task. We make calls to `getrusage()` when each job completes, measuring the total CPU time (user and system) consumed by the task since the end of the prior job. This accounts for execution of the task's function plus the overhead of context switching and timer handling. To capture worst-case conditional behavior, we force recalculation of the inversion matrix with each iteration of data inversion.

The following code reproduces the results shown in Figure 7d -- 7f.

### Obtain Execution Times

We have already provided the results data in the files under `fims/fims_execution_time_profile/result_/execution_time_on_raspberrypi`. This can be reproduced by running the following.

***Note that a single run takes around 20 minutes.***
Provided data is from 10 runs each over two different aerosol datasets. To save time for the reviewers, the below code only performs a single run by default, though this can be controlled by changing the `run_times` variable.

Input data are real images and sensor data obtained by the FIMS instrument, and are found in `fims/raw_data`.

In [ ]:
# Import Packages
import os
import sys
import time

def enter_folder(folder) :
    # print("Currently in", os.getcwd(), "entering", folder)
    if os.getcwd()[-1*len(folder):] != folder :
        os.chdir(folder)

def leave_folder(folder) :
    os.chdir("../")
    for _ in range(folder.count("/")):
        os.chdir("../")
    # print("Back in", os.getcwd())

In [ ]:
# Change this to perform additional runs
run_times = 1

# Function to rename the folder and create a new one
def manage_folders(old_folder, new_folder, arg):
    os.rename(old_folder, f"{old_folder}_{arg}")
    os.makedirs(new_folder, exist_ok=True)

execution_time_working_folder = "fims_execution_time_profile/result_"
executable = "../build/fims"  
result_folder = "run_time"

In [ ]:
# Enter execution time profiling folder
enter_folder(execution_time_working_folder)

# Create result directory
os.makedirs(result_folder, exist_ok=True)

for i in range(0, run_times, 1):  # define how many time to run the code
    print(f'start running {i} ...')
    if run_sudo(executable) == 0:
        print(f"Run {i+1}: Success")
        # Rename the old folder and create a new one for the next run
        manage_folders(result_folder, result_folder, i)
    else:
        print(f"Run {i+1}: Failure")

    time.sleep(1)  # Optional: pause for 1 second between runs

leave_folder(execution_time_working_folder)

Results are written into a collection of folders `fims/fims_execution_time_profile/result_/run_time_x`, with x indexing each run. Each file contains space-separated values corresponding to the execution time of each iteration of the corresponding task, in units of milliseconds. We use the times reported by `get_rusage()`, separately writing the system times (files ending in `_sys`) and user times (files ending in `_user`).

### Plot and Analyze Results

The following code plots Figure 7d--7f, showing execution time distributions. It also reports the worst observed execution times for each task.

We provide the ability to either plot the original data or data newly-generated by the above steps.

In [ ]:
# Import Packages
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def plot_execution_times(data, data2) :
    
    for i in range(len(data)):
        data[i] = data[i] + data2[i]

    components = ['image_processing', 'data_inversion', 'hk_reading']
    titles = ['(d) FIMS: Image', '(f) FIMS: Inversion', '(e) FIMS: HK']
    maximums = [0,0,0]

    # Define a palette
    colors = sns.color_palette("deep", 5)

    # Use the fivethirtyeight style
    plt.style.use('fivethirtyeight')

    for i, d in enumerate(data):
        fig, ax = plt.subplots(figsize=(4.6, 2.5))
        ax.hist(d, bins=30, density=True, alpha=0.8, edgecolor='black', linewidth=1.5, color=colors[i])
        ax.set_yscale('log')
        ax.set_xlabel('Execution Time (ms)', )
        ax.set_ylabel('Frequency')
        plt.tight_layout()
        fig.savefig(f'{components[i]}_execution_time.png')
        fig.savefig(f'{components[i]}_execution_time.eps')
        plt.show() 
        maximums[i] = np.max(data[i])
        print(f'{titles[i]} (max {maximums[i]}ms)')
    
    return maximums

#### Plot Original Results

Plots results obtained for the paper from the files under `fims/fims_execution_time_profile/result_/execution_time_on_raspberrypi`.

In [ ]:
# Enter execution time profiling folder
execution_time_working_folder = "fims_execution_time_profile/result_/execution_time_on_raspberrypi"
enter_folder(execution_time_working_folder)

# Process userspace time
file_name = 'sparse'
file_name2 = 'dense'
path = ['/image_process_cost_user.txt', '/inversion_time_user.txt', '/hk_reading_time_user.txt']
data = []
for p in path:
    data1 = []
    for i in range(1, 11, 1):
        file = 'run_time_' + file_name + '_' + str(i) + p
        cur_data = np.loadtxt(file)
        data1 = np.concatenate((data1, cur_data[1:]))
        if p == '/image_process_cost_user.txt':
            file = 'run_time_' + file_name2 + '_' + str(i) + p
            cur_data = np.loadtxt(file)
            data1 = np.concatenate((data1, cur_data[1:]))
    data.append(data1)

# Process system time
path_sys = ['/image_process_cost_sys.txt', '/inversion_time_sys.txt', '/hk_reading_time_sys.txt']
data2 = []
for p in path_sys:
    data1 = []
    for i in range(1, 11, 1):
        file = 'run_time_' + file_name + '_' + str(i) + p
        cur_data = np.loadtxt(file)
        data1 = np.concatenate((data1, cur_data[1:]))
        if p == '/image_process_cost_sys.txt':
            file = 'run_time_' + file_name2 + '_' + str(i) + p
            cur_data = np.loadtxt(file)
            data1 = np.concatenate((data1, cur_data[1:]))

    data2.append(data1)

# Plot
maximums = plot_execution_times(data,data2)

leave_folder(execution_time_working_folder)

#### Plot Newly-Obtained Results

Plots new results obtained from running the above profiling steps.

In [ ]:
# Enter execution time profiling folder
execution_time_working_folder = "fims_execution_time_profile/result_"
enter_folder(execution_time_working_folder)

# Process userspace time
path = ['/image_process_time_user.txt', '/inversion_time_user.txt', '/hk_reading_time_user.txt']
data = []
for p in path:
    data1 = []
    for i in range(0, run_times, 1):
        file = 'run_time_' + str(i) + p
        cur_data = np.loadtxt(file)
        data1 = np.concatenate((data1, cur_data[1:]))
    data.append(data1)

# Process system time
path_sys = ['/image_process_time_sys.txt', '/inversion_time_sys.txt', '/hk_reading_time_sys.txt']
data2 = []
for p in path_sys:
    data1 = []
    for i in range(0, run_times, 1):
        file = 'run_time_' + str(i) + p
        cur_data = np.loadtxt(file)
        data1 = np.concatenate((data1, cur_data[1:]))
    data2.append(data1)

# Plot
plot_execution_times(data,data2)

leave_folder(execution_time_working_folder)

## Assigning Elasticity Values

We next assign elasticity values by measuring the loss associated with adjusting task periods individually. To reflect the real instrument's behavior, when increasing the image processing period, we stack successive image frames; stacked images are found in `fims/raw_data/xstack`, where x denotes the number of frames to be stacked.

The following code reproduces the results shown in Figure 7a -- 7c.

### Obtain Loss Values

We have already provided the results data in the files under `fims/fims_elasticity_values_assignment/result_`. This can be reproduced by running the following.

In [ ]:
# Import Packages
import sys
import os
import json
import time

def manage_files(old_file, arg1, arg2):
    name_part, extension_part = os.path.splitext(old_file)
    new_name = f"{name_part}_{arg1}_{arg2}{extension_part}"
    os.rename(old_file, new_name)

elastic_working_folder = "fims_elasticity_values_assignment/result_"
executable = "../build/fims"  
result_file = "n_Dp_fixed.txt"
fims_config_file_path = '../configuration/config.json'

#### Image Processing

Obtain results for adjusting image processing periods from 100 to 1000 ms.

***Note: This may take around 10 minutes to run.***

In [ ]:
# Enter elasticity value folder
enter_folder(elastic_working_folder) 

# Task Period Values
T_image = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]

for i in range(len(T_image)): 
    with open(fims_config_file_path, 'r') as file:
        data = json.load(file)
        data['image_processing_duration'] = T_image[i] 
        data['HK_reading_duration'] = 500
        data['data_inversion_duration'] = 1000

    with open(fims_config_file_path, 'w') as file:
        json.dump(data, file, indent=4)

    if run_sudo(executable) == 0:
        print(f"Running program with image process duration {T_image[i]}: Success")
        manage_files(result_file, 'image', T_image[i])
    else:
        print(f"Running program with image process duration {T_image[i]}: Fail")
    time.sleep(1)  # pause for 1 second between runs

leave_folder(elastic_working_folder)

#### HK Reading

Obtain results for adjusting HK data reading periods from 500 to 5000 ms.

***Note: This may take around 30 minutes to run.***

In [ ]:
# Enter elasticity value folder
enter_folder(elastic_working_folder) 

# Task Period Values
T_HK = [500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000]

for i in range(len(T_HK)): 
    with open(fims_config_file_path, 'r') as file:
        data = json.load(file)
        data['HK_reading_duration'] = T_HK[i] 
        data['image_processing_duration'] = 100 
        data['data_inversion_duration'] = 1000

    with open(fims_config_file_path, 'w') as file:
        json.dump(data, file, indent=4)

    if run_sudo(executable) == 0:
        print(f"Running program with hk reading duration {T_HK[i]}: Success")
        manage_files(result_file, 'hk', T_HK[i])
    else:
        print(f"Running program with hk reading duration {T_HK[i]}: Fail")
    time.sleep(1)  # pause for 1 second between runs

leave_folder(elastic_working_folder)


#### Data Inversion

Obtain results for adjusting data matrix inversion periods from 1 to 10 seconds.

***Note: This may take around 30 minutes to run.***

In [ ]:
# Enter elasticity value folder
enter_folder(elastic_working_folder) 

# Task Period Values
T_inversion = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]

for i in range(len(T_inversion)): 
    with open(fims_config_file_path, 'r') as file:
        data = json.load(file)
        data['data_inversion_duration'] = T_inversion[i] 
        data['image_processing_duration'] = 100 
        data['HK_reading_duration'] = 500

    with open(fims_config_file_path, 'w') as file:
        json.dump(data, file, indent=4)

    if run_sudo(executable) == 0:
        print(f"Running program with hk reading duration {T_inversion[i]}: Success")
        manage_files(result_file, 'inversion', T_inversion[i])
    else:
        print(f"Running program with hk reading duration {T_inversion[i]}: Fail")
    time.sleep(1)  # pause for 1 second between runs

leave_folder(elastic_working_folder)

### Plot and Analyze Results

The following code plots Figure 7a--7c, deriving error for each tested task period, then fitting the error as a function of the square deviation in rate. It uses this to obtain and report the elasticity values.

#### Setup

In [ ]:
# Import Packages
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns

# Define Style for Plotting

# Use the fivethirtyeight style
plt.style.use('fivethirtyeight')

# Define a palette
colors = sns.color_palette("deep", 5)

# Read CSV files
def read_csv(file_path):
    data = []
    with open(file_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            row = list(map(float, line.strip().split(',')))
            data.append(row)
    return data

# Read Text output
def read_txt(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    matrix = []
    row = []
    for line in lines:
        value = line.strip()
        if value == "NaN":
            matrix.append(row)
            row = []
        else:
            row.append(float(value))

    if row:
        matrix.append(row)

    matrix = np.array(matrix)
    return matrix

# Skip Rows for Larger Time Windows (used for larger inversion periods)
def sum_every_n_rows(arr, n):
    num_rows_to_keep = (len(arr) // n) * n
    trimmed_arr = arr[:num_rows_to_keep]   
    reshaped = trimmed_arr.reshape(-1, n, arr.shape[1])
    return reshaped.sum(axis=1)

# Compute FIMS Errors Based on Cosine Similarities
def compute_errors(durations, n, filename) :

    enter_folder(elastic_working_folder) 

    # Get ground truth   
    n_fixed_array_matlab = [] 
    n_Dp_fixed_matlab = read_csv('n_Dp_fixed_mat.csv')
    n_Dp_fixed_matlab = np.array(n_Dp_fixed_matlab, dtype=float)
    if n > 0 :
        n_fixed_array_matlab = np.nan_to_num(n_Dp_fixed_matlab)[:n]
    else : # For larger inversion periods
        n_Dp_fixed_matlab = np.nan_to_num(n_Dp_fixed_matlab)
        for i in range(len(durations)):
            n_fixed_array_matlab_ = sum_every_n_rows(n_Dp_fixed_matlab, i+1)
            n_fixed_array_matlab.append(n_fixed_array_matlab_)

    # Get degraded result data
    n_fixed_array_cpp = []
    for i in range(len(durations)):
        n_Dp_fixed_cpp_path = filename + str(durations[i]) + '.txt'
        n_Dp_fixed_cpp = read_txt(n_Dp_fixed_cpp_path)
        n_Dp_fixed_cpp = np.array(n_Dp_fixed_cpp, dtype=float)        
        if n > 0 :
            n_Dp_fixed_cpp = np.nan_to_num(n_Dp_fixed_cpp)[:n]
        else :
            n_Dp_fixed_cpp = np.nan_to_num(n_Dp_fixed_cpp)
        n_fixed_array_cpp.append(n_Dp_fixed_cpp)

    # Compute cosine similarities for error

    n_fixed_array_matlab_norm = []
    n_fixed_array_matlab_sum = []
    
    if n > 0: # For fixed inversion periods
        n_fixed_array_matlab_sum = np.sum(n_fixed_array_matlab, axis=1)[:, np.newaxis]
        n_fixed_array_matlab_norm = np.where(n_fixed_array_matlab_sum!=0, n_fixed_array_matlab / n_fixed_array_matlab_sum, np.nan)

    n_fixed_array_cpp_norm = []
    cos_similarities = []
    mean_cos_similarities = []

    for i in range(len(durations)):
        n_fixed_array_cpp_sum = np.sum(n_fixed_array_cpp[i], axis=1)[:, np.newaxis]
        n_fixed_array_cpp_norm_ = np.where(n_fixed_array_cpp_sum!=0, n_fixed_array_cpp[i] / n_fixed_array_cpp_sum, np.nan)
        n_fixed_array_cpp_norm_ = np.nan_to_num(n_fixed_array_cpp_norm_) + 1e-15
        n_fixed_array_cpp_norm.append(n_fixed_array_cpp_norm_)

        if n > 0 : # For fixed inversion periods         
            similarities = [cosine_similarity(n_fixed_array_matlab_norm[j].reshape(1, -1), n_fixed_array_cpp_norm[i][j].reshape(1, -1))[0][0] for j in range(n_fixed_array_cpp_norm[i].shape[0])]
        else : # For larger inversion periods
            n_fixed_array_matlab_sum = np.sum(n_fixed_array_matlab[i], axis=1)[:, np.newaxis]
            n_fixed_array_matlab_norm_ = np.where(n_fixed_array_matlab_sum!=0, n_fixed_array_matlab[i] / n_fixed_array_matlab_sum, np.nan)
            n_fixed_array_matlab_norm_ = np.nan_to_num(n_fixed_array_matlab_norm_) + 1e-15
            n_fixed_array_matlab_norm.append(n_fixed_array_matlab_norm_)
            similarities = [cosine_similarity(n_fixed_array_matlab_norm[i][j].reshape(1, -1), n_fixed_array_cpp_norm[i][j].reshape(1, -1))[0][0] for j in range(n_fixed_array_cpp_norm[i].shape[0])]
            

        cos_similarities.append(similarities)
        mean_similarities = np.mean(similarities)
        mean_cos_similarities.append(mean_similarities)
    
    leave_folder(elastic_working_folder)

    return mean_cos_similarities

def fit_and_plot(mean_cos_similarities, wcet) :

    # Fit error to square of rate deviation

    R = 1 / durations * 1000
    R_max = R[0]
    X = (R_max - R) ** 2
    Y = (mean_cos_similarities[0] - mean_cos_similarities) * 1000

    X = np.array(X)
    Y = np.array(Y)

    slope, intercept = np.polyfit(X, Y, 1)

    # Plot

    plt.figure(figsize=(4.6, 2.5))
    plt.scatter(X, Y, color='blue')
    plt.plot(X, slope * X + intercept, color='red', label='Best Fit')
    plt.xlabel(r'$(R^{max} - R)^2$') 
    plt.ylabel('Error')
    plt.tight_layout()
    plt.legend()
    plt.show()

    # Compute Elasticity
    e = wcet ** 2 / slope / 1000
    print(f'Elasticity Constant: {e}')
    return e

def derive_elasticity(durations, n, filename, wcet) :    
    return fit_and_plot(compute_errors(durations, n, filename), wcet)




#### Figure 7a

The following code plots Figure 7a, showing the derived error for each image processing period (from 100--1000) and fitting as a function of square deviation in rate. It also reports the derived elasticity according to Equation 23.

In [ ]:
durations = np.array([100, 200, 300, 400, 500, 600, 700, 800, 900, 1000])
n = 1193
filename = "n_Dp_fixed_image_"
wcet_image = maximums[0]
e_image = derive_elasticity(durations, n, filename, wcet_image)

#### Figure 7b

The following code plots Figure 7b, showing the derived error for each HK data reading period (from 500--5000) and fitting as a function of square deviation in rate. It also reports the derived elasticity according to Equation 23.

In [ ]:
durations = np.array([500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000])
n = 1195
filename = "n_Dp_fixed_hk_"
wcet_hk = maximums[2]
e_hk = derive_elasticity(durations, n, filename, wcet_hk)

#### Figure 7c

The following code plots Figure 7c, showing the derived error for each data inversion period (from 1000--10000) and fitting as a function of square deviation in rate. It also reports the derived elasticity according to Equation 23. Note that this is more involved because ... TODO

In [ ]:
durations = np.array([1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000])
n = 0
filename = "n_Dp_fixed_inversion_"
wcet_inversion = maximums[1]
e_inversion = derive_elasticity(durations, n, filename, wcet_inversion)

## Evaluating Scalability

To test under tighter utilization constraints, we run a highest priority interference task that limits CPU utilization by FIMS: it registers an interval timer with a period of 50 ms and spins for a programmable length of time. We run FIMS concurrently with the interference task using different busy loop durations, adjusting the FIMS task periods according to our harmonic elastic model. We then measure each FIMS job’s latency (elapsed wallclock time) from task release to completion.

The following code reproduces the results shown in the last table of Section VI.B.

### Obtain Periods According to Elastic Model

First, build the wrapper program around the elastic harmonic library to obtain the periods:

In [ ]:
%%sh
cd ..
make fims_periods

Then execute, supplying the WCET and E values already obtained:

In [ ]:
command = f"../fims_periods {wcet_image} {e_image} {wcet_hk} {e_hk} {wcet_inversion} {e_inversion}"
os.system(command)

### Run Scalability Experiments

We have already provided the results data in the files under `fims/fims_evaluate_scalability/result_/result_on_raspberrypi`. This can be reproduced by running the following.

First, update periods according to the assigned values from the elastic model, based on above output:

In [ ]:
T_image = [100, 115, 147, 222, 458]
T_HK = [500, 575, 881, 3325, 3205]
T_inv = [1000, 2298, 9682, 9973, 9615]

Then, run FIMS alongside the interference process.

***Note: This may take around 2 hours to run.***

In [ ]:
# Import Packages
import subprocess
import os
import signal
import time
import psutil
import json

# Kill subprocess and children
def kill_process_tree(pid, including_parent=True):  
    parent = psutil.Process(pid)
    for child in parent.children(recursive=True):
        child.kill()
    if including_parent:
        parent.kill()

# Function to run the C++ program
def run_cpp_programs(executable1, executable2, args2):
    cmd1 = f"echo {password} | sudo -S {executable1}"
    cmd2 = f"echo {password} | sudo -S {executable2} {args2}"

    proc1 = subprocess.Popen(cmd1, shell=True)
    proc2 = subprocess.Popen(cmd2, shell=True)

    # Wait for proc1 to complete
    proc1.communicate()

    # Kill proc2 when proc1 completes
    kill_process_tree(proc2.pid)

    return proc1.returncode == 0

# Function to rename the folder and create a new one
def manage_folders(old_folder, new_folder, arg):
    # Rename the folder
    os.rename(old_folder, f"{old_folder}_{arg}")

    # Create a new folder for the next run
    os.makedirs(new_folder, exist_ok=True)


scalability_working_folder = "fims_evaluate_scalability/result_"
fims_executable = "../build/fims"
overhead_executable = '../../over_head/overhead'
overhead_exe = 'overhead'
result_folder = 'run_time'
fims_config_file_path = '../configuration/config.json'

# Utilizations to test
utilization_D = [0.5, 0.4, 0.3, 0.2, 0.1]

enter_folder(scalability_working_folder)

#choose the dataset you want to run by changing the range
for i in range(len(utilization_D)): 
    with open(fims_config_file_path, 'r') as file:
        data = json.load(file)

    data['HK_reading_duration'] = T_HK[i]  
    data['image_processing_duration'] = T_image[i] 
    data['data_inversion_duration'] = T_inv[i] 

    with open(fims_config_file_path, 'w') as file:
        json.dump(data, file, indent=4)
    
    overhead_utilization = 1 - utilization_D[i]
    if run_cpp_programs(fims_executable, overhead_executable, str(overhead_utilization)):
        print(f"Running program with available utilization {utilization_D[i]}: Success")
        manage_folders(result_folder, result_folder, utilization_D[i])
    else:
        print(f"Running program with available utilization {utilization_D[i]}: Failure")


    time.sleep(1)  # pause for 1 second between runs

leave_folder(scalability_working_folder)


### Analyze Results

The following parses the data in `fims/fims_evaluate_scalability/result_/result_on_raspberrypi` to produce the results shown in the last table of Section VI.B.

In [ ]:
enter_folder("fims_evaluate_scalability/result_/result_on_raspberrypi")

utilizations = [0.1, 0.2, 0.3, 0.4, 0.5]
components=['/image_process_cost.txt', '/hk_reading_time.txt', '/inversion_time.txt']

data = []
for component in components:
    component_data = []
    for utilization in utilizations:
        data1 = []
        for i in range(1, 4, 1):
            file = 'run_time_' + str(utilization) + '_' + str(i) + component
            cur_data = np.loadtxt(file)
            data1 = np.concatenate((data1, cur_data[1:]))
        component_data.append(data1)
    data.append(component_data)


components=['image_process', 'hk_reading', 'data_inversion']
for i in range(len(data)):
    for j in range(len(data[i])):
        maximum = np.max(data[i][j])
        print(f'{components[i]} utilization {utilizations[j]} Max Time: {maximum}')

leave_folder("fims_evaluate_scalability/result_/result_on_raspberrypi")
